In [53]:
import vaex
for i, df in enumerate(vaex.read_csv('./data/Taxi_Trips_-_2017.csv', chunk_size=100_000)):
    df.export_hdf5(f'./chunks/taxi_{i:02}.hdf5')

In [56]:
df_taxi_trips_all = vaex.open('./chunks/taxi_*.hdf5')
df_taxi_trips_all.head()

#,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,7ce98ec3d6d2b81bf6a91a5333e84690ec4e94f3,'30ac065db3b331d97b0842f0cb1740ed5b8d7fe26530441...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.5,nan,1.70311e+10,nan,8,7.5,0,0,1.5,9,Cash,Dispatch Taxi Affiliation,nan,nan,--,41.9059,-87.6309,POINT (-87.6308650266 41.9058577688)
1,e447e5c2e41c4f6751907d5e686f35b34e315afb,'4e902eb187e3942c1da821c8aa349fc848c6ff3f71f1070...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,27,0,0,0,27,Credit Card,Suburban Dispatch LLC,nan,nan,--,nan,nan,--
2,95e39db90a6a468a863f43c8eccce7e26ba3db39,'78a7a28fe001257044d5095fcb1ff7156a47411ff93cc5e...,01/01/2017 12:00:00 AM,01/01/2017 12:45:00 AM,2280,0.2,nan,nan,nan,76,22.25,0,0,0,22.25,Cash,Dispatch Taxi Affiliation,nan,nan,--,41.9803,-87.9136,POINT (-87.913624596 41.9802643146)
3,b09914181a9bd6a5dcd2e44941d215bc663763df,'c157870bbd395a664de1f3e6f7868b3b89472295c4e2d6e...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.7,nan,nan,nan,nan,7.5,0,0,0,7.5,Cash,4787 - 56058 Reny Cab Co,nan,nan,--,nan,nan,--
4,d039b0e5600d5008f95bd55114fe8acdeba81d99,'b5c4dfe366c83cef5604db1afcfbc15c8c38eeefd896b98...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,540,0,nan,nan,nan,nan,6.75,0,0,1,7.75,Cash,Blue Ribbon Taxi Association Inc.,nan,nan,--,nan,nan,--
5,94291f34710a6ee268b32bd3da208f6cbb65a0de,'ff6aa87ad2487541184808bdb9202867f5ec8ff5db6adf5...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,6.25,0,0,0,6.25,Credit Card,Suburban Dispatch LLC,nan,nan,--,nan,nan,--
6,f00546971a3e4c8ab55761a337443a0dfa33f42f,'631de3a4ad35b7d7d75949b33ad59e09820b51b3f58225d...,01/01/2017 12:00:00 AM,01/01/2017 12:30:00 AM,1800,0,1.70311e+10,1.70311e+10,8,8,13.25,4.25,0,1,18.5,Credit Card,Taxi Affiliation Services,41.8932,-87.6378,POINT (-87.6378442095 41.8932163595),41.8925,-87.6262,POINT (-87.6262149064 41.8925077809)
7,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,660,2.7,nan,nan,2,3,10,2,0,0,12,Credit Card,Choice Taxi Association,42.0016,-87.695,POINT (-87.6950125892 42.001571027),41.9658,-87.6559,POINT (-87.6558787862 41.96581197)
8,42b662e1238bb7dceccca2b19d395df44dc12dd5,'5985b42dd0cbd4492cd51df71c1fd5dbed60edb96d589b7...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,7.5,3,0,0,10.5,Credit Card,Suburban Dispatch LLC,nan,nan,--,nan,nan,--
9,89b4ff599f5d23fa177b8bd915b7aa3c2e0608b3,'29051a8f04056bb47ab86564c9fec32eb9e15e8ce6aed24...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,21.75,3.25,0,0,25,Credit Card,T.A.S. - Payment Only,nan,nan,--,nan,nan,--


In [57]:
df_taxi_trips_all.describe()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
data_type,string,string,string,string,float64,float64,float64,float64,float64,float64,...,float64,float64,string,string,float64,float64,string,float64,float64,string
count,24988003,24979611,24988003,24987341,24986557,24987519,16937859,16851806,22686142,22230677,...,24987415,24987415,24988003,24988003,22690605,22690605,22690605,22301841,22301841,22301841
NA,0,8392,0,662,1446,484,8050144,8136197,2301861,2757326,...,588,588,0,0,2297398,2297398,2297398,2686162,2686162,2686162
mean,--,--,--,--,811.9719720888316,3.6095682328461693,17031394502.071043,17031370762.653587,24.78923216649177,22.37094943172446,...,1.0144652221928492,15.964313623496194,--,--,41.89957355462784,-87.66040122216353,--,41.89985113310513,-87.65445607029591,--
std,--,--,--,--,1113.482617,5.826476,342227.623152,330454.206498,20.189024,18.045409,...,12.259628,32.620101,--,--,0.039178,0.076272,--,0.038754,0.061783,--
min,--,--,--,--,0.0,0.0,17031010100.0,17031010100.0,1.0,1.0,...,0.0,0.0,--,--,41.660136,-87.913625,--,41.660136,-87.913625,--
max,--,--,--,--,86389.0,2151.86,17031980100.0,17031980100.0,77.0,77.0,...,9864.84,9999.99,--,--,42.021224,-87.52995,--,42.021224,-87.52995,--


In [93]:
column_names = df_taxi_trips_all.column_names
column_names_refactored = [ln.replace(' ', '_').lower() for ln in column_names]

for i, column in enumerate(column_names):
    df_taxi_trips_all.rename(column, column_names_refactored[i])

df_taxi_trips_all.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,7ce98ec3d6d2b81bf6a91a5333e84690ec4e94f3,'30ac065db3b331d97b0842f0cb1740ed5b8d7fe26530441...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.5,nan,1.70311e+10,nan,8,7.5,0,0,1.5,9,Cash,Dispatch Taxi Affiliation,nan,nan,41.9059,-87.6309
1,e447e5c2e41c4f6751907d5e686f35b34e315afb,'4e902eb187e3942c1da821c8aa349fc848c6ff3f71f1070...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,27,0,0,0,27,Credit Card,Suburban Dispatch LLC,nan,nan,nan,nan
2,95e39db90a6a468a863f43c8eccce7e26ba3db39,'78a7a28fe001257044d5095fcb1ff7156a47411ff93cc5e...,01/01/2017 12:00:00 AM,01/01/2017 12:45:00 AM,2280,0.2,nan,nan,nan,76,22.25,0,0,0,22.25,Cash,Dispatch Taxi Affiliation,nan,nan,41.9803,-87.9136
3,b09914181a9bd6a5dcd2e44941d215bc663763df,'c157870bbd395a664de1f3e6f7868b3b89472295c4e2d6e...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.7,nan,nan,nan,nan,7.5,0,0,0,7.5,Cash,4787 - 56058 Reny Cab Co,nan,nan,nan,nan
4,d039b0e5600d5008f95bd55114fe8acdeba81d99,'b5c4dfe366c83cef5604db1afcfbc15c8c38eeefd896b98...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,540,0,nan,nan,nan,nan,6.75,0,0,1,7.75,Cash,Blue Ribbon Taxi Association Inc.,nan,nan,nan,nan
5,94291f34710a6ee268b32bd3da208f6cbb65a0de,'ff6aa87ad2487541184808bdb9202867f5ec8ff5db6adf5...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,6.25,0,0,0,6.25,Credit Card,Suburban Dispatch LLC,nan,nan,nan,nan
6,f00546971a3e4c8ab55761a337443a0dfa33f42f,'631de3a4ad35b7d7d75949b33ad59e09820b51b3f58225d...,01/01/2017 12:00:00 AM,01/01/2017 12:30:00 AM,1800,0,1.70311e+10,1.70311e+10,8,8,13.25,4.25,0,1,18.5,Credit Card,Taxi Affiliation Services,41.8932,-87.6378,41.8925,-87.6262
7,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,660,2.7,nan,nan,2,3,10,2,0,0,12,Credit Card,Choice Taxi Association,42.0016,-87.695,41.9658,-87.6559
8,42b662e1238bb7dceccca2b19d395df44dc12dd5,'5985b42dd0cbd4492cd51df71c1fd5dbed60edb96d589b7...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,7.5,3,0,0,10.5,Credit Card,Suburban Dispatch LLC,nan,nan,nan,nan
9,89b4ff599f5d23fa177b8bd915b7aa3c2e0608b3,'29051a8f04056bb47ab86564c9fec32eb9e15e8ce6aed24...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,0,0,nan,nan,nan,nan,21.75,3.25,0,0,25,Credit Card,T.A.S. - Payment Only,nan,nan,nan,nan


In [94]:
# count amount of community areas
print(len(df_taxi_trips_all['pickup_community_area'].unique()))
print(len(df_taxi_trips_all['dropoff_community_area'].unique()))

78
78


In [95]:
# count amount of census tracts
print(len(df_taxi_trips_all['pickup_census_tract'].unique()))
print(len(df_taxi_trips_all['dropoff_census_tract'].unique()))


953
1075


In [96]:
df_taxi_trips_all.drop(['dropoff_centroid__location', 'pickup_centroid_location'])

KeyError: "Column or variable 'dropoff_centroid__location' does not exist. Did you mean: 'dropoff_centroid_latitude' or 'dropoff_centroid_longitude' or 'dropoff_census_tract'"

In [97]:
# drop rows with trip_seconds = 0
df_taxi_trips_all_remove_0_sec = df_taxi_trips_all[df_taxi_trips_all['trip_seconds'] != 0.0]
df_taxi_trips_all_remove_0_sec.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,7ce98ec3d6d2b81bf6a91a5333e84690ec4e94f3,'30ac065db3b331d97b0842f0cb1740ed5b8d7fe26530441...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.5,nan,1.70311e+10,nan,8,7.5,0,0,1.5,9,Cash,Dispatch Taxi Affiliation,nan,nan,41.9059,-87.6309
1,95e39db90a6a468a863f43c8eccce7e26ba3db39,'78a7a28fe001257044d5095fcb1ff7156a47411ff93cc5e...,01/01/2017 12:00:00 AM,01/01/2017 12:45:00 AM,2280,0.2,nan,nan,nan,76,22.25,0,0,0,22.25,Cash,Dispatch Taxi Affiliation,nan,nan,41.9803,-87.9136
2,b09914181a9bd6a5dcd2e44941d215bc663763df,'c157870bbd395a664de1f3e6f7868b3b89472295c4e2d6e...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.7,nan,nan,nan,nan,7.5,0,0,0,7.5,Cash,4787 - 56058 Reny Cab Co,nan,nan,nan,nan
3,d039b0e5600d5008f95bd55114fe8acdeba81d99,'b5c4dfe366c83cef5604db1afcfbc15c8c38eeefd896b98...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,540,0,nan,nan,nan,nan,6.75,0,0,1,7.75,Cash,Blue Ribbon Taxi Association Inc.,nan,nan,nan,nan
4,f00546971a3e4c8ab55761a337443a0dfa33f42f,'631de3a4ad35b7d7d75949b33ad59e09820b51b3f58225d...,01/01/2017 12:00:00 AM,01/01/2017 12:30:00 AM,1800,0,1.70311e+10,1.70311e+10,8,8,13.25,4.25,0,1,18.5,Credit Card,Taxi Affiliation Services,41.8932,-87.6378,41.8925,-87.6262
5,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,660,2.7,nan,nan,2,3,10,2,0,0,12,Credit Card,Choice Taxi Association,42.0016,-87.695,41.9658,-87.6559
6,149f18c2d8b9bce6001cc9e156092aa562c83bc1,'b7a562fe2d37014bb47274cb5baa361b97ff9263e3ba526...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,120,0.5,nan,nan,16,16,4.5,0,0,0,4.5,Cash,Taxi Affiliation Services,41.9536,-87.7235,41.9536,-87.7235
7,beaec50c0923d1dbe1217a507342eefdef81e39b,'7f83e923d71df8812353e1ea8f491ca1cc70469fe9ed276...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,960,2.7,nan,nan,nan,nan,12,0,0,0,12,Cash,City Service,nan,nan,nan,nan
8,7d863c35284952ed19ce5d5d946e6946c710a15a,'bfb5c2ad40103c9891e7ee4713e93ccd75e741099b74988...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,600,1.7,nan,nan,nan,nan,8.25,2,0,0,10.75,Credit Card,Chicago Independents,nan,nan,nan,nan
9,16dc1e4922ed3375f78fa1e418f52b4178bc21a9,'3c814d3baedca9be4de8ddb5547c7dec404a08e148740ee...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,720,1.7,nan,nan,nan,nan,8.75,0,0,1,9.75,Cash,Sun Taxi,nan,nan,nan,nan


In [117]:
df_taxi_trips_all_remove_0_sec_miles = df_taxi_trips_all_remove_0_sec[df_taxi_trips_all_remove_0_sec['trip_miles'] != 0.0]
df_taxi_trips_all_remove_0_sec_miles.head()

#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,7ce98ec3d6d2b81bf6a91a5333e84690ec4e94f3,'30ac065db3b331d97b0842f0cb1740ed5b8d7fe26530441...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.5,nan,1.70311e+10,nan,8,7.5,0,0,1.5,9,Cash,Dispatch Taxi Affiliation,nan,nan,41.9059,-87.6309
1,95e39db90a6a468a863f43c8eccce7e26ba3db39,'78a7a28fe001257044d5095fcb1ff7156a47411ff93cc5e...,01/01/2017 12:00:00 AM,01/01/2017 12:45:00 AM,2280,0.2,nan,nan,nan,76,22.25,0,0,0,22.25,Cash,Dispatch Taxi Affiliation,nan,nan,41.9803,-87.9136
2,b09914181a9bd6a5dcd2e44941d215bc663763df,'c157870bbd395a664de1f3e6f7868b3b89472295c4e2d6e...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,480,1.7,nan,nan,nan,nan,7.5,0,0,0,7.5,Cash,4787 - 56058 Reny Cab Co,nan,nan,nan,nan
3,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,660,2.7,nan,nan,2,3,10,2,0,0,12,Credit Card,Choice Taxi Association,42.0016,-87.695,41.9658,-87.6559
4,149f18c2d8b9bce6001cc9e156092aa562c83bc1,'b7a562fe2d37014bb47274cb5baa361b97ff9263e3ba526...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,120,0.5,nan,nan,16,16,4.5,0,0,0,4.5,Cash,Taxi Affiliation Services,41.9536,-87.7235,41.9536,-87.7235
5,beaec50c0923d1dbe1217a507342eefdef81e39b,'7f83e923d71df8812353e1ea8f491ca1cc70469fe9ed276...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,960,2.7,nan,nan,nan,nan,12,0,0,0,12,Cash,City Service,nan,nan,nan,nan
6,7d863c35284952ed19ce5d5d946e6946c710a15a,'bfb5c2ad40103c9891e7ee4713e93ccd75e741099b74988...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,600,1.7,nan,nan,nan,nan,8.25,2,0,0,10.75,Credit Card,Chicago Independents,nan,nan,nan,nan
7,16dc1e4922ed3375f78fa1e418f52b4178bc21a9,'3c814d3baedca9be4de8ddb5547c7dec404a08e148740ee...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,720,1.7,nan,nan,nan,nan,8.75,0,0,1,9.75,Cash,Sun Taxi,nan,nan,nan,nan
8,891d3f4ddf25e499e60c3c7bf4c715a005d588fe,'5045e67af9f438ea9a9b2ffc8c084ac86b9ebe8df69619d...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,780,5.9,nan,nan,nan,nan,16.75,3.45,0,0,20.7,Credit Card,Taxi Affiliation Service Yellow,nan,nan,nan,nan
9,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,'56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,240,1.3,1.70318e+10,1.70318e+10,32,28,6.25,0,0,0,6.25,Cash,Taxi Affiliation Services,41.881,-87.6327,41.8853,-87.6572


In [108]:
df_taxi_trips_all_remove_0_sec_miles.describe()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
data_type,string,string,string,string,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,string,string,float64,float64,float64,float64
count,21938478,21936454,21938478,21938478,21937919,21938004,15004859,14937706,20135507,19718757,...,21937997,15532303,21937997,21937997,21938478,21938478,20136918,20136918,19784195,19784195
NA,0,2024,0,0,559,474,6933619,7000772,1802971,2219721,...,481,6406175,481,481,0,0,1801560,1801560,2154283,2154283
mean,--,--,--,--,859.9978368960155,4.103767578857101,17031393331.624466,17031366939.506407,24.84262924196545,22.19605180995942,...,1.6382144021623513,0.0025626714853553634,0.9941212472588082,16.62984985549015,--,--,41.89933029500626,-87.66039721485849,41.89958801119896,-87.65371144389488
std,--,--,--,--,942.85128,6.022518,343184.781388,332390.541959,20.191795,17.808416,...,2.848026,0.630514,7.37093,27.571928,--,--,0.038885,0.07669,0.038271,0.060688
min,--,--,--,--,1.0,0.01,17031010100.0,17031010100.0,1.0,1.0,...,0.0,0.0,0.0,0.0,--,--,41.660136,-87.913625,41.660136,-87.913625
max,--,--,--,--,86374.0,2151.86,17031980100.0,17031980100.0,77.0,77.0,...,800.0,1024.99,7230.66,9999.99,--,--,42.021224,-87.534903,42.021224,-87.531386


In [123]:
df_taxi_trips_all_remove_0_sec_miles.dropnan(column_names=["taxi_id", "trip_seconds", "trip_miles", "pickup_centroid_latitude", "pickup_centroid_longitude", "dropoff_centroid_latitude", "dropoff_centroid_longitude"], how='any')


#,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
0,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,'b7da4d1247a408c100b4964f236760694290aace75c8b87...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,660.0,2.7,nan,nan,2.0,3.0,10.0,2.0,0.0,0.0,12.0,Credit Card,Choice Taxi Association,42.001571027,-87.695012589,41.96581197,-87.655878786
1,149f18c2d8b9bce6001cc9e156092aa562c83bc1,'b7a562fe2d37014bb47274cb5baa361b97ff9263e3ba526...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,120.0,0.5,nan,nan,16.0,16.0,4.5,0.0,0.0,0.0,4.5,Cash,Taxi Affiliation Services,41.953582125,-87.72345239,41.953582125,-87.72345239
2,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,'56a1119c6ca57e39525cf06829f9ecff553cf4b5ac24821...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,240.0,1.3,17031839100.0,17031833000.0,32.0,28.0,6.25,0.0,0.0,0.0,6.25,Cash,Taxi Affiliation Services,41.880994471,-87.632746489,41.88528132,-87.6572332
3,c3ee40051c5643c54064a8b9df3686d2d411518d,'7987cfe5e3b2c9869665f70fe35dbe7423e7fc5852f2d6f...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,1500.0,0.4,17031839100.0,17031081300.0,32.0,8.0,22.75,4.85,0.0,1.5,29.1,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994471,-87.632746489,41.898331794,-87.620762865
4,29f30c8a6bbc028f5d63aa18c708d31b47267a9f,'0b269137bd94b55212311226d9e1554423e12bcc16a432c...,01/01/2017 12:00:00 AM,01/01/2017 12:30:00 AM,1620.0,0.9,17031839100.0,17031081500.0,32.0,8.0,12.25,2.85,0.0,2.0,17.1,Credit Card,Taxi Affiliation Services,41.880994471,-87.632746489,41.892507781,-87.626214906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"19,574,959",c93361b0a17ff369829b395bf3b6bda145bfbfed,'b24c87ca4c0f737a91884bfea7ffa458374128765621c10...,05/20/2017 07:00:00 PM,05/20/2017 07:15:00 PM,668.0,1.1,17031320100.0,17031081700.0,32.0,8.0,7.75,0.0,0.0,1.5,9.75,Credit Card,City Service,41.884987192,-87.620992913,41.892042136,-87.63186395
"19,574,960",4e9b8ab53980635cd3db48fcaa8b6ced1e5d97b9,'7b50110eaa441d38f0dba8a06ee13d5cb2854661aafb856...,05/20/2017 07:00:00 PM,05/20/2017 07:15:00 PM,840.0,2.0,17031071500.0,17031081500.0,7.0,8.0,10.0,0.0,0.0,1.0,11.5,Credit Card,Medallion Leasin,41.914616286,-87.631717366,41.892507781,-87.626214906
"19,574,961",65181efbc9772989fa55d206e26da9787cd9d384,'72b3babcea62dbec800fef8b2989cfb563f7ed2fc7de034...,05/20/2017 07:00:00 PM,05/20/2017 07:30:00 PM,1980.0,17.3,17031081403.0,17031081403.0,8.0,8.0,44.25,9.95,0.0,5.0,59.7,Credit Card,Medallion Leasin,41.890922026,-87.618868355,41.890922026,-87.618868355
"19,574,962",32fafbdfefa40acd034c4ee555750d8a898fa358,'a02e1f355041b6350e4d79a4de1e650b266f5dc1ea40d19...,05/20/2017 07:00:00 PM,05/20/2017 07:15:00 PM,780.0,0.7,17031081600.0,17031320400.0,8.0,32.0,7.75,0.0,0.0,0.0,7.75,Cash,Globe Taxi,41.892072635,-87.628874157,41.877406123,-87.621971652


In [124]:
df_taxi_trips_all_remove_0_sec_miles.describe()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
data_type,string,string,string,string,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,string,string,float64,float64,float64,float64
count,21938478,21936454,21938478,21938478,21937919,21938004,15004859,14937706,20135507,19718757,...,21937997,15532303,21937997,21937997,21938478,21938478,20136918,20136918,19784195,19784195
NA,0,2024,0,0,559,474,6933619,7000772,1802971,2219721,...,481,6406175,481,481,0,0,1801560,1801560,2154283,2154283
mean,--,--,--,--,859.9978368960155,4.103767578857656,17031393331.624466,17031366939.506407,24.84262924196545,22.19605180995942,...,1.6382144021623513,0.0025626714853553634,0.9941212472588082,16.629849855490153,--,--,41.89933029499875,-87.66039721485843,41.89958801120455,-87.65371144388062
std,--,--,--,--,942.85128,6.022518,343184.733646,332390.541959,20.191795,17.808416,...,2.848026,0.630514,7.37093,27.571928,--,--,0.038885,0.07669,0.038271,0.060689
min,--,--,--,--,1.0,0.01,17031010100.0,17031010100.0,1.0,1.0,...,0.0,0.0,0.0,0.0,--,--,41.660136,-87.913625,41.660136,-87.913625
max,--,--,--,--,86374.0,2151.86,17031980100.0,17031980100.0,77.0,77.0,...,800.0,1024.99,7230.66,9999.99,--,--,42.021224,-87.534903,42.021224,-87.531386


In [112]:
#df_taxi_trips_all_remove_0_sec_miles_no_longitude_latitude.describe()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude
data_type,string,string,string,string,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,string,string,float64,float64,float64,float64
count,19574964,19572999,19574964,19574964,19574964,19574964,14843789,14843789,19573705,19512889,...,19574935,14118616,19574935,19574935,19574964,19574964,19574964,19574964,19574964,19574964
NA,0,1965,0,0,0,0,4731175,4731175,1259,62075,...,29,5456348,29,29,0,0,0,0,0,0
mean,--,--,--,--,829.5955623724263,3.71404277371502,17031388640.569298,17031365192.715197,23.98560778350343,21.974223191655526,...,1.582495195002749,0.0007643518316526209,0.7723220450029584,15.864478992646557,--,--,41.898129903810904,-87.65606940331112,41.899186698673546,-87.65250822301194
std,--,--,--,--,905.035709,5.409563,340679.304073,331609.439673,19.246827,17.485542,...,2.662956,0.311844,7.297541,26.385352,--,--,0.037228,0.070434,0.037627,0.058482
min,--,--,--,--,1.0,0.01,17031010100.0,17031010100.0,1.0,1.0,...,0.0,0.0,0.0,0.0,--,--,41.660136,-87.913625,41.660136,-87.913625
max,--,--,--,--,86374.0,1567.2,17031980100.0,17031980100.0,77.0,77.0,...,800.0,1024.99,7230.66,9826.37,--,--,42.021224,-87.534903,42.021224,-87.531386


In [101]:
#len(df_taxi_trips_all)

24988003

In [102]:
#ölen(df_taxi_trips_all_remove_0_sec_miles_no_longitude_latitude)

19575796